<h1 align="center">The Battle of Neighborhoods</h1>

This notebook is the final Capstone project of the IBM Data Science Professional Certification. It explores neighborhoods in Toronto, Canada and applies machine learning to provide a recommendation on best locations for opening a Japanese restaurant

<h2>Part 1 - Scrape Wikipedia and create Postal Code dataframe</h2>

<p>Before getting data and start exploring it, download all dependencies that will be needed.</p>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c anaconda beautifulsoup4
from bs4 import BeautifulSoup # library to scrape websites

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                       

### Scrape wikipedia to get postal code table data

In [2]:
res = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(res, 'lxml')
my_table = soup.find('table',{'class':'wikitable'})
    
postalCode=[]
borough=[]
neighborhood=[]
for row in my_table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells)==3:
        postalCode.append(cells[0].find(text=True))
        borough.append(cells[1].find(text=True))
        neighborhood.append(cells[2].find(text=True))      

### Transform the data into a *pandas* dataframe.

In [3]:
df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])
df['PostalCode'] = postalCode
df['Borough'] = borough
df['Neighborhood'] = neighborhood

### Cleanse the data

In [4]:
# Ignore cells with a borough that is Not assigned
neighborhoods = df[~df['Borough'].str.contains('Not assigned')].reset_index(drop=True)

# Combine rows with same PostalCode, separate neighborhoods by comma
neighborhoods.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(','.join).reset_index()

# If cell has a borough but neighborhood is Not Assigned, set Neighborhood to be same as the borough.
neighborhoods['Neighborhood'] = np.where((neighborhoods['Borough'] != '') & (neighborhoods['Neighborhood'].str.contains('Not assigned')), neighborhoods['Borough'], neighborhoods['Neighborhood'])

# Get rid of new line characters in PostalCode column
neighborhoods['PostalCode'] = neighborhoods['PostalCode'].replace('\n','', regex=True)

neighborhoods

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
# Print shape of dataframe
neighborhoods.shape

(103, 3)

<h1>Part 2 - Create geographical coordinates dataframe using Geocoder</h1>

In [6]:
!conda install -c conda-forge geocoder
import geocoder # import geocoder

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

    geocoder:        1.38.1-py_1       conda-forge
    ratelim:         0.1.6-py_2        conda-forge

The following packages will be UPDATED:

    ca-certificates: 2020.1.1-0        anaconda    --> 2020.4.5.1-hecc5488_0     conda-forge
    certifi:         2020.4.5.1-py36_0 anaconda    --> 2020.4.5.1-py36h9f0ad1d_0 conda-forge

The following packages will be DOWNGRADED:

    ope

### Create a function to get latitude and longitude of a given postal code

In [ ]:
def getGeoCoordinates(postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude, longitude

### Loop through all neighborhoods and call above function to get coordinates. Add Latitude and Longitude columns to dataframe

In [9]:
# Loop through dataframe to get geographical coordinates for each postal code
latitudes=[]
longitudes=[]
for index, row in neighborhoods.iterrows():
    postalCode = str(row['PostalCode'])
    latitude, longitude = getGeoCoordinates(postalCode)
    latitudes.append(latitude)
    longitudes.append(longitude)
 
# Add Latitude and Longitude columns to dataframe
neighborhoods_merged = neighborhoods
neighborhoods_merged['Latitude'] = latitudes
neighborhoods_merged['Longitude'] = longitudes

neighborhoods_merged

KeyboardInterrupt: 

### Since Geocoder Python package can be unreliable, run the below steps if above does not give results

In [10]:
!wget -O Geospatial_Coordinates.csv http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv

--2020-04-13 15:56:15--  http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Resolving cocl.us (cocl.us)... 158.85.108.86, 169.48.113.194, 158.85.108.83
Connecting to cocl.us (cocl.us)|158.85.108.86|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv [following]
--2020-04-13 15:56:15--  https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Connecting to cocl.us (cocl.us)|158.85.108.86|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-04-13 15:56:17--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197, 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9af

In [11]:
geoCoordinates = pd.read_csv('Geospatial_Coordinates.csv')
geoCoordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
# Join neighborhoods and geoCoordinates dataframes to add Latitude and Longitude columns
neighborhoods_merged = neighborhoods
neighborhoods_merged = neighborhoods_merged.join(geoCoordinates.set_index('Postal Code'), on='PostalCode')
neighborhoods_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


<h1>Part 3 - Explore and Cluster Neighborhoods in Toronto</h1>

### Create a Map of Toronto to visualize all neighborhoods

In [13]:
# latitude and longitude of Toronto
latitude = 43.6532
longitude = -79.3832

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods_merged['Latitude'], neighborhoods_merged['Longitude'], neighborhoods_merged['Borough'], neighborhoods_merged['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Filter boroughs that contain the word Toronto

In [14]:
toronto_data = neighborhoods_merged[neighborhoods_merged['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,Richmond / Adelaide / King,43.650571,-79.384568
9,M6H,West Toronto,Dufferin / Dovercourt Village,43.669005,-79.442259


### Define Foursquare Credentials and Version

In [15]:
CLIENT_ID = '544JX2O4RUIRPRZHNXUEW5PFB1NXPFR5V2RF4OCGDZ0ZJR4V' # your Foursquare ID
CLIENT_SECRET = 'KXCB2PFFGR3TI0AOCLGU4NDHQDFI5Z4MNBCGCVPIJZ2WF31C' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 544JX2O4RUIRPRZHNXUEW5PFB1NXPFR5V2RF4OCGDZ0ZJR4V
CLIENT_SECRET:KXCB2PFFGR3TI0AOCLGU4NDHQDFI5Z4MNBCGCVPIJZ2WF31C


### Explore Neighborhoods in Toronto

#### Create a function that gets top 100 venues within a radius of 500 metres for  a given neighborhood

In [16]:
LIMIT = 100
radius = 500

# Method returns nearby venus for a given neighborhoood
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [17]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park / Harbourfront

Queen's Park / Ontario Provincial Government

Garden District, Ryerson

St. James Town

The Beaches

Berczy Park

Central Bay Street

Christie

Richmond / Adelaide / King

Dufferin / Dovercourt Village

Harbourfront East / Union Station / Toronto Islands

Little Portugal / Trinity

The Danforth West / Riverdale

Toronto Dominion Centre / Design Exchange

Brockton / Parkdale Village / Exhibition Place

India Bazaar / The Beaches West

Commerce Court / Victoria Hotel

Studio District

Lawrence Park

Roselawn

Davisville North

Forest Hill North & West

High Park / The Junction South

North Toronto West

The Annex / North Midtown / Yorkville

Parkdale / Roncesvalles

Davisville

University of Toronto / Harbord

Runnymede / Swansea

Moore Park / Summerhill East

Kensington Market / Chinatown / Grange Park

Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park

CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst

Rosedale

S

In [18]:
print(toronto_venues.shape)
toronto_venues.head()

(1622, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park / Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park / Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park / Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Regent Park / Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Regent Park / Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [19]:
toronto_venues.groupby(['Neighborhood']).count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
Brockton / Parkdale Village / Exhibition Place,23,23,23,23,23,23
Business reply mail Processing CentrE,16,16,16,16,16,16
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst,14,14,14,14,14,14
Central Bay Street,64,64,64,64,64,64
Christie,18,18,18,18,18,18
Church and Wellesley,75,75,75,75,75,75
Commerce Court / Victoria Hotel,100,100,100,100,100,100
Davisville,34,34,34,34,34,34


In [20]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 227 uniques categories.


### Analyze Each Neighborhood

In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot.drop(['Neighborhood'], axis=1, inplace=True)
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [22]:
toronto_onehot.shape

(1622, 227)

In [23]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.018182,0.000000,0.000000,0.000000,0.0000,0.018182,0.000000,0.018182,0.036364,0.000000,0.000000,0.00,0.018182,0.018182,0.000000,0.036364,0.000000,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,0.036364,0.000000,0.000000,0.000000,0.036364,0.000000,0.000000,0.000000,0.000000,0.000000,0.036364,0.054545,0.000000,0.00000,0.000000,0.00,0.000000,0.018182,0.000000,0.018182,0.000000,0.018182,0.000,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,0.000000,0.00,0.000000,0.000000,0.036364,0.000000,0.000000,0.000,0.018182,

In [24]:
toronto_grouped.shape

(39, 227)

#### Print each neighborhood along with the top 5 most common venues

In [25]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park
----
            venue  freq
0     Coffee Shop  0.05
1  Farmers Market  0.04
2    Cocktail Bar  0.04
3      Restaurant  0.04
4        Beer Bar  0.04


----Brockton / Parkdale Village / Exhibition Place
----
            venue  freq
0            Café  0.13
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3          Bakery  0.04
4       Pet Store  0.04


----Business reply mail Processing CentrE
----
            venue  freq
0      Comic Shop  0.06
1   Garden Center  0.06
2            Park  0.06
3      Restaurant  0.06
4  Farmers Market  0.06


----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst
----
                venue  freq
0      Airport Lounge  0.14
1     Airport Service  0.14
2    Airport Terminal  0.14
3             Airport  0.07
4  Airport Food Court  0.07


----Central Bay Street
----
                venue  freq
0         Coffee Shop  0.19
1                Café  0.06
2  Italian Restaurant  0.06
3      Sandwich Place  0.05
4     Bubble Tea

#### Create a function to sort the venues in descending order.

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create the new dataframe and display the top 10 venues for each neighborhood.

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Farmers Market,Cheese Shop,Beer Bar,Bakery,Café,Restaurant,Italian Restaurant,Seafood Restaurant,Cocktail Bar
1,Brockton / Parkdale Village / Exhibition Place,Café,Breakfast Spot,Coffee Shop,Grocery Store,Bakery,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
2,Business reply mail Processing CentrE,Skate Park,Pizza Place,Butcher,Burrito Place,Fast Food Restaurant,Farmers Market,Auto Workshop,Spa,Restaurant,Garden
3,CN Tower / King and Spadina / Railway Lands / ...,Airport Lounge,Airport Service,Airport Terminal,Airport,Airport Food Court,Airport Gate,Boutique,Harbor / Marina,Boat or Ferry,Sculpture Garden
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Spa,Ice Cream Shop,Japanese Restaurant,Sushi Restaurant,Bubble Tea Shop,Salad Place


### Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [28]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [139]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# some cleansing is needed
toronto_merged = toronto_merged[toronto_merged['Cluster Labels'].notna()]
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype('int32')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Breakfast Spot,Café,Restaurant,Theater,Shoe Store
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,0,Coffee Shop,Diner,Fried Chicken Joint,Burrito Place,Juice Bar,Sandwich Place,Café,Italian Restaurant,Beer Bar,Mexican Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Bookstore,Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cocktail Bar,Restaurant,Beer Bar,American Restaurant,Hotel,Italian Restaurant,Seafood Restaurant,Park
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


Finally, we visualize the resulting clusters

In [140]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine the clusters

In [141]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Regent Park / Harbourfront,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Breakfast Spot,Café,Restaurant,Theater,Shoe Store
1,Queen's Park / Ontario Provincial Government,Coffee Shop,Diner,Fried Chicken Joint,Burrito Place,Juice Bar,Sandwich Place,Café,Italian Restaurant,Beer Bar,Mexican Restaurant
2,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Bookstore,Restaurant
3,St. James Town,Coffee Shop,Café,Cocktail Bar,Restaurant,Beer Bar,American Restaurant,Hotel,Italian Restaurant,Seafood Restaurant,Park
4,The Beaches,Health Food Store,Trail,Pub,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
5,Berczy Park,Coffee Shop,Farmers Market,Cheese Shop,Beer Bar,Bakery,Café,Restaurant,Italian Restaurant,Seafood Restaurant,Cocktail Bar
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Spa,Ice Cream Shop,Japanese Restaurant,Sushi Restaurant,Bubble Tea Shop,Salad Place
7,Christie,Grocery Store,Café,Park,Coffee Shop,Nightclub,Italian Restaurant,Candy Store,Baby Store,Athletics & Sports,Restaurant
8,Richmond / Adelaide / King,Coffee Shop,Café,Restaurant,Gym,Thai Restaurant,Hotel,Deli / Bodega,Bar,Sushi Restaurant,Pizza Place
9,Dufferin / Dovercourt Village,Pharmacy,Bakery,Grocery Store,Recording Studio,Café,Supermarket,Middle Eastern Restaurant,Bar,Bank,Brewery


In [142]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Rosedale,Park,Playground,Trail,Yoga Studio,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [143]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,Home Service,Garden,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [144]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,Park,Swim School,Bus Line,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
21,Forest Hill North & West,Park,Jewelry Store,Trail,Sushi Restaurant,Bus Line,Diner,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [145]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Moore Park / Summerhill East,Playground,Trail,Tennis Court,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


<h1>Part 4 - Provide recommendations on locations for a Japanese restaurant</h1>

<p>Now that we have applied Clustering to neighborhoods and explored the neighborhoods, we will use the results to further refine the neighborhoods. Several factors need to be studied to recommend a location:
<ul>
    <li>Whether location is a prime or busy location.
    <li>Restaurant competition in the neighborhoods – are there any existing Japanese or Sushi restaurants nearby?</li>
    <li>Would a Japanese restaurant “fit in” with the location – e.g. we wouldn’t want to open a Japanese restaurant next to a freeway, rail, airport or around fast food restaurants.</li>
    <li>Nearby attractions – how accessible is it to potential customers.</li>
</ul>
</p>

### Determine whether location is a prime or busy location.

In [146]:
# Find which cluster has the most venues - neighborhoods in that cluster are considered as prime locations
prime_flags = np.zeros(len(toronto_merged))

primeCluster = -1
maxVenues = -1
for k in range(kclusters):
    cluster_num_rows = len(toronto_merged[toronto_merged['Cluster Labels'] == k])
    print('Number of rows in cluster ' + str(k) + ': ' + str(cluster_num_rows))
    if cluster_num_rows > maxVenues: 
        maxVenues = cluster_num_rows
        primeCluster = k

print('Prime cluster is: ' + str(primeCluster) + ' with number of venues = ' + str(maxVenues))
prime_flags = toronto_merged['Cluster Labels'].apply(lambda x: 1 if  x == primeCluster else 0)
toronto_merged['PrimeLocation'] = prime_flags
toronto_merged[['Neighborhood', 'Cluster Labels', 'PrimeLocation']]

Number of rows in cluster 0: 34
Number of rows in cluster 1: 1
Number of rows in cluster 2: 1
Number of rows in cluster 3: 2
Number of rows in cluster 4: 1
Prime cluster is: 0 with number of venues = 34


,Neighborhood,Cluster Labels,PrimeLocation
0,Regent Park / Harbourfront,0,1
1,Queen's Park / Ontario Provincial Government,0,1
2,"Garden District, Ryerson",0,1
3,St. James Town,0,1
4,The Beaches,0,1
5,Berczy Park,0,1
6,Central Bay Street,0,1
7,Christie,0,1
8,Richmond / Adelaide / King,0,1
9,Dufferin / Dovercourt Village,0,1


### Explore Restaurant Competition in the neighborhoods

In [199]:
# Find which neighborhoods already has a Japanese or Sushi restaurant in their top 10 venues
import re

low_competition_flags = np.zeros(len(toronto_merged))

# Select only the top 10 most common venues columns
mostCommonVenues = toronto_merged.iloc[:, 6:16]
#mostCommonVenues

# Create a dataframe with True/False values. If cell value in mostCommonVenues contains 'Japanese Restaurant' or 'Sushi Restaurant', set to True else False
hasJapaneseOrSushiFlags = mostCommonVenues.apply(lambda x: x.str.contains('Japanese Restaurant|Sushi Restaurant', flags=re.IGNORECASE))
hasJapaneseOrSushi = hasJapaneseOrSushiFlags.any(axis=1)

# Set low_competition_flags - Set to 1 if there are no Japanese or Sushi restaurant, else set to 0
low_competition_flags = hasJapaneseOrSushi.apply(lambda x: 0 if x == True else 1)
toronto_merged['LowCompetition'] = low_competition_flags
toronto_merged.loc[:, toronto_merged.columns[[2] + list(range(6, 16)) + [17]]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,LowCompetition
0,Regent Park / Harbourfront,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Breakfast Spot,Café,Restaurant,Theater,Shoe Store,1
1,Queen's Park / Ontario Provincial Government,Coffee Shop,Diner,Fried Chicken Joint,Burrito Place,Juice Bar,Sandwich Place,Café,Italian Restaurant,Beer Bar,Mexican Restaurant,1
2,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Bookstore,Restaurant,0
3,St. James Town,Coffee Shop,Café,Cocktail Bar,Restaurant,Beer Bar,American Restaurant,Hotel,Italian Restaurant,Seafood Restaurant,Park,1
4,The Beaches,Health Food Store,Trail,Pub,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,1
5,Berczy Park,Coffee Shop,Farmers Market,Cheese Shop,Beer Bar,Bakery,Café,Restaurant,Italian Restaurant,Seafood Restaurant,Cocktail Bar,1
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Spa,Ice Cream Shop,Japanese Restaurant,Sushi Restaurant,Bubble Tea Shop,Salad Place,0
7,Christie,Grocery Store,Café,Park,Coffee Shop,Nightclub,Italian Restaurant,Candy Store,Baby Store,Athletics & Sports,Restaurant,1
8,Richmond / Adelaide / King,Coffee Shop,Café,Restaurant,Gym,Thai Restaurant,Hotel,Deli / Bodega,Bar,Sushi Restaurant,Pizza Place,0
9,Dufferin / Dovercourt Village,Pharmacy,Bakery,Grocery Store,Recording Studio,Café,Supermarket,Middle Eastern Restaurant,Bar,Bank,Brewery,1


### Would a Japanese restaurant “fit in” with the location? Make sure there are no freeways, railway stations, airports or fast food restaurants

In [200]:
location_fit_flags = np.zeros(len(toronto_merged))

# Select only the top 10 most common venues columns
mostCommonVenues = toronto_merged.iloc[:, 6:16]
#mostCommonVenues

# Create a dataframe with True/False values. If cell value in mostCommonVenues contains 'railway' or 'freeway' or 'aiport' or 'Fast Food Restaurant', set to True else False
hasUnwantedVenueFlags = mostCommonVenues.apply(lambda x: x.str.contains('railway|freeway|airport|fast food', flags=re.IGNORECASE))
hasUnwantedVenue = hasUnwantedVenueFlags.any(axis=1)

# Set low_competition_flags - Set to 1 if entire row is False (i.e there are no Japanese or Sushi restaurant), else set to 0

location_fit_flags = hasUnwantedVenue.apply(lambda x: 0 if x == True else 1)
toronto_merged['LocationFit'] = location_fit_flags
toronto_merged.loc[:, toronto_merged.columns[[2] + list(range(6, 16)) + [18]]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,LocationFit
0,Regent Park / Harbourfront,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Breakfast Spot,Café,Restaurant,Theater,Shoe Store,1
1,Queen's Park / Ontario Provincial Government,Coffee Shop,Diner,Fried Chicken Joint,Burrito Place,Juice Bar,Sandwich Place,Café,Italian Restaurant,Beer Bar,Mexican Restaurant,1
2,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Bookstore,Restaurant,1
3,St. James Town,Coffee Shop,Café,Cocktail Bar,Restaurant,Beer Bar,American Restaurant,Hotel,Italian Restaurant,Seafood Restaurant,Park,1
4,The Beaches,Health Food Store,Trail,Pub,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,1
5,Berczy Park,Coffee Shop,Farmers Market,Cheese Shop,Beer Bar,Bakery,Café,Restaurant,Italian Restaurant,Seafood Restaurant,Cocktail Bar,1
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Spa,Ice Cream Shop,Japanese Restaurant,Sushi Restaurant,Bubble Tea Shop,Salad Place,1
7,Christie,Grocery Store,Café,Park,Coffee Shop,Nightclub,Italian Restaurant,Candy Store,Baby Store,Athletics & Sports,Restaurant,1
8,Richmond / Adelaide / King,Coffee Shop,Café,Restaurant,Gym,Thai Restaurant,Hotel,Deli / Bodega,Bar,Sushi Restaurant,Pizza Place,1
9,Dufferin / Dovercourt Village,Pharmacy,Bakery,Grocery Store,Recording Studio,Café,Supermarket,Middle Eastern Restaurant,Bar,Bank,Brewery,1


### Nearby attractions – how accessible is it to potential customers. Check for parks, hotels, tourist attrations, scenic lookouts - these are signs of accessibility or better settings for our restaurant

In [201]:
location_accessible_flags = np.zeros(len(toronto_merged))

# Select only the top 10 most common venues columns
mostCommonVenues = toronto_merged.iloc[:, 6:16]
#mostCommonVenues

# Create a dataframe with True/False values. If cell value in mostCommonVenues contains 'railway' or 'freeway' or 'aiport' or 'Fast Food Restaurant', set to True else False
hasAccessibleVenueFlags = mostCommonVenues.apply(lambda x: x.str.contains('Park|Hotel|Trail|Aquarium|Museum|Theater|Scenic Lookout', flags=re.IGNORECASE))
hasAccessibleVenue = hasAccessibleVenueFlags.any(axis=1)

# Set low_competition_flags - Set to 1 if entire row is False (i.e there are no Japanese or Sushi restaurant), else set to 0

location_accessible_flags = hasAccessibleVenue.apply(lambda x: 1 if x == True else 0)
toronto_merged['LocationAccessible'] = location_accessible_flags
toronto_merged.loc[:, toronto_merged.columns[[2] + list(range(6, 16)) + [19]]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,LocationAccessible
0,Regent Park / Harbourfront,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Breakfast Spot,Café,Restaurant,Theater,Shoe Store,1
1,Queen's Park / Ontario Provincial Government,Coffee Shop,Diner,Fried Chicken Joint,Burrito Place,Juice Bar,Sandwich Place,Café,Italian Restaurant,Beer Bar,Mexican Restaurant,0
2,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Bookstore,Restaurant,0
3,St. James Town,Coffee Shop,Café,Cocktail Bar,Restaurant,Beer Bar,American Restaurant,Hotel,Italian Restaurant,Seafood Restaurant,Park,1
4,The Beaches,Health Food Store,Trail,Pub,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,1
5,Berczy Park,Coffee Shop,Farmers Market,Cheese Shop,Beer Bar,Bakery,Café,Restaurant,Italian Restaurant,Seafood Restaurant,Cocktail Bar,0
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Spa,Ice Cream Shop,Japanese Restaurant,Sushi Restaurant,Bubble Tea Shop,Salad Place,0
7,Christie,Grocery Store,Café,Park,Coffee Shop,Nightclub,Italian Restaurant,Candy Store,Baby Store,Athletics & Sports,Restaurant,1
8,Richmond / Adelaide / King,Coffee Shop,Café,Restaurant,Gym,Thai Restaurant,Hotel,Deli / Bodega,Bar,Sushi Restaurant,Pizza Place,1
9,Dufferin / Dovercourt Village,Pharmacy,Bakery,Grocery Store,Recording Studio,Café,Supermarket,Middle Eastern Restaurant,Bar,Bank,Brewery,0


### Build final observation table with a Recommend: Yes/No column

In [202]:
# add up the 1's to get a score.  Recommend the ones with a score of 4
toronto_merged['Score'] = toronto_merged[['PrimeLocation', 'LowCompetition', 'LocationFit', 'LocationAccessible']].sum(axis=1)
toronto_merged['Recommend: Yes/No'] = toronto_merged['Score'].apply(lambda x: 'Yes' if x == 4 else 'No')
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PrimeLocation,LowCompetition,LocationFit,LocationAccessible,Score,Recommend: Yes/No
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Breakfast Spot,Café,Restaurant,Theater,Shoe Store,1,1,1,1,4,Yes
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,0,Coffee Shop,Diner,Fried Chicken Joint,Burrito Place,Juice Bar,Sandwich Place,Café,Italian Restaurant,Beer Bar,Mexican Restaurant,1,1,1,0,3,No
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Bookstore,Restaurant,1,0,1,0,2,No
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cocktail Bar,Restaurant,Beer Bar,American Restaurant,Hotel,Italian Restaurant,Seafood Restaurant,Park,1,1,1,1,4,Yes
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,1,1,1,1,4,Yes
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Farmers Market,Cheese Shop,Beer Bar,Bakery,Café,Restaurant,Italian Restaurant,Seafood Restaurant,Cocktail Bar,1,1,1,0,3,No
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Spa,Ice Cream Shop,Japanese Restaurant,Sushi Restaurant,Bubble Tea Shop,Salad Place,1,0,1,0,2,No
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Coffee Shop,Nightclub,Italian Restaurant,Candy Store,Baby Store,Athletics & Sports,Restaurant,1,1,1,1,4,Yes
8,M5H,Downtown Toronto,Richmond / Adelaide / King,43.650571,-79.384568,0,Coffee Shop,Café,Restaurant,Gym,Thai Restaurant,Hotel,Deli / Bodega,Bar,Sushi Restaurant,Pizza Place,1,0,1,1,3,No
9,M6H,West Toronto,Dufferin / Dovercourt Village,43.669005,-79.442259,0,Pharmacy,Bakery,Grocery Store,Recording Studio,Café,Supermarket,Middle Eastern Restaurant,Bar,Bank,Brewery,1,1,1,0,3,No


In [158]:
# These are the recommended locations
toronto_merged[toronto_merged['Recommend: Yes/No'] == 'Yes']

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PrimeLocation,LowCompetition,LocationFit,LocationAccessible,Score,Recommend: Yes/No
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Breakfast Spot,Café,Restaurant,Theater,Shoe Store,1,1,1,1,4,Yes
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cocktail Bar,Restaurant,Beer Bar,American Restaurant,Hotel,Italian Restaurant,Seafood Restaurant,Park,1,1,1,1,4,Yes
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,1,1,1,1,4,Yes
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Coffee Shop,Nightclub,Italian Restaurant,Candy Store,Baby Store,Athletics & Sports,Restaurant,1,1,1,1,4,Yes
10,M5J,Downtown Toronto,Harbourfront East / Union Station / Toronto Is...,43.640816,-79.381752,0,Coffee Shop,Aquarium,Hotel,Restaurant,Italian Restaurant,Café,Brewery,Sporting Goods Shop,Scenic Lookout,Fried Chicken Joint,1,1,1,1,4,Yes
20,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Park,Breakfast Spot,Gym,Food & Drink Shop,Sandwich Place,Department Store,Hotel,Discount Store,Ethiopian Restaurant,Electronics Store,1,1,1,1,4,Yes
24,M5R,Central Toronto,The Annex / North Midtown / Yorkville,43.672710,-79.405678,0,Café,Sandwich Place,Coffee Shop,Pharmacy,Indian Restaurant,BBQ Joint,History Museum,Pizza Place,Pub,American Restaurant,1,1,1,1,4,Yes
25,M6R,West Toronto,Parkdale / Roncesvalles,43.648960,-79.456325,0,Gift Shop,Bookstore,Movie Theater,Eastern European Restaurant,Italian Restaurant,Dog Run,Bar,Bank,Restaurant,Dessert Shop,1,1,1,1,4,Yes
35,M4X,Downtown Toronto,St. James Town / Cabbagetown,43.667967,-79.367675,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Pub,Bakery,Pizza Place,Park,Sandwich Place,Butcher,1,1,1,1,4,Yes


In [192]:
# create map of clusters, but adding a Tick Marker on the recommended locations
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, recommend in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels'], toronto_merged['Recommend: Yes/No']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
      
    if recommend == 'Yes':
        folium.Marker(location=[lat,lon],
                      popup = str(poi) + ': Highly Recommended!', 
                      icon= folium.Icon(color='Red', 
                      icon_color='green',icon = 'check')).add_to(map_clusters) 

map_clusters